In [2]:
# Install YOLOv5
# Install YOLOv5 in the GitHub folder
%cd /export/home1/users/zfne/emanuell/Documents/GitHub
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt

/export/home1/users/zfne/emanuell/Documents/GitHub
Cloning into 'yolov5'...
remote: Enumerating objects: 16605, done.
remote: Counting objects: 100% (146/146), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 16605 (delta 50), reused 96 (delta 43), pack-reused 16459
Receiving objects: 100% (16605/16605), 15.32 MiB | 22.42 MiB/s, done.
Resolving deltas: 100% (11349/11349), done.
/export/home1/users/zfne/emanuell/Documents/GitHub/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 2.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of scipy to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 43.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 2.9 MB/s eta 

In [7]:
import yaml
# Define the classes
classes = ['dolphin', 'fish', 'diver']

# Define the data dictionary
data_yaml = {
    'train': '/media/DOLPHIN_ALEXIS/Analyses_alexis/CSDNC2/images/',
    'val': '/media/DOLPHIN_ALEXIS/Analyses_alexis/CSDNC2/images/',
    'nc': len(classes),
    'names': classes
}

# Write the data dictionary to the data.yaml file
with open('/media/DOLPHIN_ALEXIS/Analyses_alexis/CSDNC2/data_yaml.yaml', 'w') as f:
    yaml.dump(data_yaml, f)


In [8]:
# Train the model
# Train the model
!python train.py --img 640 --batch 16 --epochs 100 --data /media/DOLPHIN_ALEXIS/Analyses_alexis/CSDNC2/data_yaml.yaml --cfg yolov5s.yaml --weights yolov5s.pt

2024-06-11 14:35:46.601601: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-11 14:35:46.601635: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-11 14:35:46.602716: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=yolov5s.yaml, data=/media/DOLPHIN_ALEXIS/Analyses_alexis/CSDNC2/data_yaml.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_wei

In [ ]:
# Evaluate the model
!python val.py --data /kaggle/working/data_yaml.yaml --weights runs/train/exp/weights/best.pt

In [25]:
import cv2
import torch
from models.common import DetectMultiBackend
from utils.general import non_max_suppression

# Check if GPU is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

# Load YOLOv5 model
device = torch.device(device)
model = DetectMultiBackend('/users/zfne/emanuell/Documents/GitHub/yolov5/runs/train/exp3/weights/last.pt', device=device)
model.eval()

# Define a function to resize frames to the model's expected size
def resize_frame_to_model_size(frame, model_stride=32):
    height, width = frame.shape[:2]
    new_height, new_width = (height // model_stride) * model_stride, (width // model_stride) * model_stride
    resized_frame = cv2.resize(frame, (new_width, new_height))
    return resized_frame

# Define a function to process video frames
def process_frame(frame, model):
    # Resize the frame to ensure it is compatible with the model
    frame = resize_frame_to_model_size(frame)
    
    # Preprocess the frame for YOLOv5
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = img / 255.0
    img = torch.from_numpy(img).float().to(device).permute(2, 0, 1).unsqueeze(0)

    # Run YOLOv5 on the frame
    with torch.no_grad():
        pred = model(img)

    # Apply NMS
    pred = non_max_suppression(pred)[0]
    
    # Extract bounding boxes and labels
    boxes = pred.cpu().numpy()
    labels = model.names
    
    return boxes, labels

# Define a function to annotate frames with detected bounding boxes
def annotate_frame(frame, boxes, labels):
    for box in boxes:
        x1, y1, x2, y2, conf, cls = box
        if labels[int(cls)] == 'diver' or labels[int(cls)] == 'dolphin' or labels[int(cls)] == 'fish:
            # Draw bounding box
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            # Add label
            cv2.putText(frame, f'{labels[int(cls)]} {conf:.2f}', (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    return frame

# Load the video file
video_path = '/media/DOLPHIN_ALEXIS/2023/Exp_01_Jun_2024_1145_cam1-4.mp4'
cap = cv2.VideoCapture(video_path)

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create VideoWriter object
output_path = '/media/DOLPHIN_ALEXIS/output_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# Check if video loaded successfully
if not cap.isOpened():
    print("Error opening video stream or file")

# Process the video frame by frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    # Ensure frame is in the correct format
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the entire frame
    boxes, labels = process_frame(frame_rgb, model)

    # Annotate the frame with the detected boxes and labels
    annotated_frame = annotate_frame(frame, boxes, labels)

    # Write the annotated frame to the output video file
    out.write(annotated_frame)
    
# Release video capture and writer objects
cap.release()
out.release()

print("Video processing complete. The output video is saved to /media/DOLPHIN_ALEXIS/output_video.mp4")

Fusing layers... 
YOLOv5s summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs


Using device: cuda
Video processing complete. The output video is saved to /media/DOLPHIN_ALEXIS/output_video.mp4
